In [1]:
import os
import sys
import argparse
import time
import faiss
from dotenv import load_dotenv
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from langchain_community.docstore.in_memory import InMemoryDocstore
load_dotenv()

True

In [4]:
pip install faiss-cpu

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/14.9 MB ? eta -:--:--
   ------- -------------------------------- 2.9/14.9 MB 28.2 MB/s eta 0:00:01
   --------- ------------------------------ 3.7/14.9 MB 8.7 MB/s eta 0:00:02
   -------------- ------------------------- 5.5/14.9 MB 9.9 MB/s eta 0:00:01
   ----------------- ---------------------- 6.6/14.9 MB 7.7 MB/s eta 0:00:02
   -------------------- ------------------- 7.6/14.9 MB 7.2 MB/s eta 0:00:02
   ----------------------- ---------------- 8.7/14.9 MB 7.1 MB/s eta 0:00:01
   ------------------------- -------------- 9.4/14.9 MB 6.4 MB/s eta 0:00:01
   --------------------------- ------------ 10.2/14.9 MB 6.1 MB/s eta 0:00:01
   ------------------------------ --------- 11.3/14.9 MB 5.8 MB/s eta 0:00:01
   --------------------------------- ------ 12.3/14.9 MB 5.8 MB/s eta 0:00:01
   ------------------------------------ --- 13.6/14.9 MB 5.8 MB/s eta 0:00:01


In [2]:
# Add the parent directory to the path since we work with notebooks
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [3]:
from helper_functions import *
from evaluation.rag_evaluation import *

C:\Users\Revathi\AppData\Local\Temp\ipykernel_16032\1040137016.py:1: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from helper_functions import *


In [4]:
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

In [5]:
class HyPE:
    """
    A class to handle the HyPE RAG process, which enhances document chunking by 
    generating hypothetical questions as proxies for retrieval.
    """

    def __init__(self, path, chunk_size=1000, chunk_overlap=200, n_retrieved=3):
        """
        Initializes the HyPE-based RAG retriever by encoding the PDF document with 
        hypothetical prompt embeddings.

        Args:
            path (str): Path to the PDF file to encode.
            chunk_size (int): Size of each text chunk (default: 1000).
            chunk_overlap (int): Overlap between consecutive chunks (default: 200).
            n_retrieved (int): Number of chunks to retrieve for each query (default: 3).
        """
        print("\n--- Initializing HyPE RAG Retriever ---")

        # Encode the PDF document into a FAISS vector store using hypothetical prompt embeddings
        start_time = time.time()
        self.vector_store = self.encode_pdf(path, chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        self.time_records = {'Chunking': time.time() - start_time}
        print(f"Chunking Time: {self.time_records['Chunking']:.2f} seconds")

        # Create a retriever from the vector store
        self.chunks_query_retriever = self.vector_store.as_retriever(search_kwargs={"k": n_retrieved})
    def generate_hypothetical_prompt_embeddings(self, chunk_text):
        """
        Uses an LLM to generate multiple hypothetical questions for a single chunk.
        These questions act as 'proxies' for the chunk during retrieval.

        Parameters:
        chunk_text (str): Text contents of the chunk.

        Returns:
        tuple: (Original chunk text, List of embedding vectors generated from the questions)
        """
        llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")
        embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

        question_gen_prompt = PromptTemplate.from_template(
            "Analyze the input text and generate essential questions that, when answered, \
            capture the main points of the text. Each question should be one line, \
            without numbering or prefixes.\n\n \
            Text:\n{chunk_text}\n\nQuestions:\n"
        )
        question_chain = question_gen_prompt | llm | StrOutputParser()

        # Parse questions from response
        questions = question_chain.invoke({"chunk_text": chunk_text}).replace("\n\n", "\n").split("\n")

        return chunk_text, embedding_model.embed_documents(questions)

    def prepare_vector_store(self, chunks):
        """
        Creates and populates a FAISS vector store using hypothetical prompt embeddings.

        Parameters:
        chunks (List[str]): A list of text chunks to be embedded and stored.

        Returns:
        FAISS: A FAISS vector store containing the embedded text chunks.
        """
        vector_store = None  # Wait to initialize to determine vector size

        with ThreadPoolExecutor() as pool:
            # Parallelized embedding generation
            futures = [pool.submit(self.generate_hypothetical_prompt_embeddings, c) for c in chunks]

            for f in tqdm(as_completed(futures), total=len(chunks)):  
                chunk, vectors = f.result()  # Retrieve processed chunk and embeddings

                # Initialize FAISS store once vector size is known
                if vector_store is None:
                    vector_store = FAISS(
                        embedding_function=OpenAIEmbeddings(model="text-embedding-3-small"),
                        index=faiss.IndexFlatL2(len(vectors[0])),
                        docstore=InMemoryDocstore(),
                        index_to_docstore_id={}
                    )

                # Store multiple vector representations per chunk
                chunks_with_embedding_vectors = [(chunk.page_content, vec) for vec in vectors]
                vector_store.add_embeddings(chunks_with_embedding_vectors)

        return vector_store

    def encode_pdf(self, path, chunk_size=1000, chunk_overlap=200):
        """
        Encodes a PDF document into a vector store using hypothetical prompt embeddings.

        Args:
            path: The path to the PDF file.
            chunk_size: The size of each text chunk.
            chunk_overlap: The overlap between consecutive chunks.

        Returns:
            A FAISS vector store containing the encoded book content.
        """
        # Load PDF documents
        loader = PyPDFLoader(path)
        documents = loader.load()

        # Split documents into chunks
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len
        )
        texts = text_splitter.split_documents(documents)
        cleaned_texts = replace_t_with_space(texts)

        return self.prepare_vector_store(cleaned_texts)

    def run(self, query):
        """
        Retrieves and displays the context for the given query.

        Args:
            query (str): The query to retrieve context for.

        Returns:
            None
        """
        # Measure retrieval time
        start_time = time.time()
        context = retrieve_context_per_question(query, self.chunks_query_retriever)
        self.time_records['Retrieval'] = time.time() - start_time
        print(f"Retrieval Time: {self.time_records['Retrieval']:.2f} seconds")

        # Deduplicate context and display results
        context = list(set(context))
        show_context(context)


In [6]:
def run_and_return_context(hype_obj, query):
    """
    Run HyPE RAG retrieval for a query and return the deduplicated context chunks.
    """
    print(f"\n Running query: {query}\n")
    start_time = time.time()
    context = retrieve_context_per_question(query, hype_obj.chunks_query_retriever)
    retrieval_time = time.time() - start_time
    print(f"⏱ Retrieval Time: {retrieval_time:.2f} seconds")

    context = list(set(context))  # remove duplicates

    for i, chunk in enumerate(context):
        print(f"\n--- Chunk {i+1} ---\n{chunk}")

    return context


In [7]:
def evaluate_rag_queries(hype_obj, queries_with_answers):
    """
    Evaluate HyPE RAG by checking if expected answers appear in retrieved context.
    
    Parameters:
        hype_obj: HyPE object
        queries_with_answers: List of tuples (query, expected_answer_snippet)
    """
    total = len(queries_with_answers)
    hits = []

    print(f"\n Running evaluation on {total} queries...\n")

    for i, (query, expected_substring) in enumerate(queries_with_answers, 1):
        print(f"\n Query {i}: {query}")
        retrieved_context = run_and_return_context(hype_obj, query)
        retrieved_text = " ".join(retrieved_context).lower()

        hit = expected_substring.lower() in retrieved_text
        hits.append(hit)

        print(f" Match found: {hit}")
        print(f"Expected snippet: \"{expected_substring}\"\n")

    accuracy = sum(hits) / total
    print(f"\n Final Accuracy: {accuracy * 100:.2f}%")

    return hits


In [9]:
# Simulated CLI args for notebook
args = argparse.Namespace(
    path=r"C:\Users\Revathi\Documents\GenAIProjects\All_RAG_Techniques\data\Understanding_Climate_Change.pdf",
    chunk_size=1000,
    chunk_overlap=200,
    n_retrieved=3,
    query="What is the main cause of climate change?",
    evaluate=False
)

# Initialize HyPE
hyperag = HyPE(
    path=args.path,
    chunk_size=args.chunk_size,
    chunk_overlap=args.chunk_overlap,
    n_retrieved=args.n_retrieved
)

# Run a single query
retrieved_context = run_and_return_context(hyperag, args.query)


--- Initializing HyPE RAG Retriever ---


100%|██████████████████████████████████████████████████████████████████████████████████| 97/97 [02:39<00:00,  1.64s/it]
C:\Users\Revathi\Documents\GenAIProjects\All_RAG_Techniques\helper_functions.py:143: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = chunks_query_retriever.get_relevant_documents(question)


Chunking Time: 162.37 seconds

 Running query: What is the main cause of climate change?

⏱ Retrieval Time: 0.79 seconds

--- Chunk 1 ---
Most of these climate changes are attributed to very small variations in Earth's orbit that 
change the amount of solar energy our planet receives. During the Holocene epoch, which 
began at the end of the last ice age, human societies flourished, but the industrial era has seen 
unprecedented changes. 
Modern Observations 
Modern scientific observations indicate a rapid increase in global temperatures, sea levels, 
and extreme weather events. The Intergovernmental Panel on Climate Change (IPCC) has 
documented these changes extensively. Ice core samples, tree rings, and ocean sediments 
provide a historical record that scientists use to understand past climate conditions and 
predict future trends. The evidence overwhelmingly shows that recent changes are primarily 
driven by human activities, particularly the emission of greenhouse gases. 
Chapter 

In [10]:
def validate_args(args):
    if args.chunk_size <= 0:
        raise ValueError("chunk_size must be a positive integer.")
    if args.chunk_overlap < 0:
        raise ValueError("chunk_overlap must be a non-negative integer.")
    if args.n_retrieved <= 0:
        raise ValueError("n_retrieved must be a positive integer.")
    return args

In [11]:
def parse_args():
    parser = argparse.ArgumentParser(description="Encode a PDF document and test a HyPE-based RAG system.")
    parser.add_argument("--path", type=str, default="../data/Understanding_Climate_Change.pdf",
                        help="Path to the PDF file to encode.")
    parser.add_argument("--chunk_size", type=int, default=1000,
                        help="Size of each text chunk (default: 1000).")
    parser.add_argument("--chunk_overlap", type=int, default=200,
                        help="Overlap between consecutive chunks (default: 200).")
    parser.add_argument("--n_retrieved", type=int, default=3,
                        help="Number of chunks to retrieve for each query (default: 3).")
    parser.add_argument("--query", type=str, default="What is the main cause of climate change?",
                        help="Query to test the retriever (default: 'What is the main cause of climate change?').")
    parser.add_argument("--evaluate", action="store_true",
                        help="Whether to evaluate the retriever's performance (default: False).")

    return validate_args(parser.parse_args())

In [13]:
query_answer_pairs = [
    ("What is the main cause of climate change?", "burning of fossil fuels"),
    ("What are greenhouse gases?", "carbon dioxide and methane"),
    ("How does deforestation affect climate?", "reduces carbon absorption"),
]

hits = evaluate_rag_queries(hyperag, query_answer_pairs)


 Running evaluation on 3 queries...


 Query 1: What is the main cause of climate change?

 Running query: What is the main cause of climate change?

⏱ Retrieval Time: 0.47 seconds

--- Chunk 1 ---
Most of these climate changes are attributed to very small variations in Earth's orbit that 
change the amount of solar energy our planet receives. During the Holocene epoch, which 
began at the end of the last ice age, human societies flourished, but the industrial era has seen 
unprecedented changes. 
Modern Observations 
Modern scientific observations indicate a rapid increase in global temperatures, sea levels, 
and extreme weather events. The Intergovernmental Panel on Climate Change (IPCC) has 
documented these changes extensively. Ice core samples, tree rings, and ocean sediments 
provide a historical record that scientists use to understand past climate conditions and 
predict future trends. The evidence overwhelmingly shows that recent changes are primarily 
driven by human activit